<a href="https://colab.research.google.com/github/IamMoShi/GTI771-IA-TPs/blob/main/Lab1_GTI771_J24_Ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTI771 - Apprentissage machine avancé
## Département de génie logiciel et des technologies de l’information (LogTI)



## Laboratoire 1 - Préparation des données
#### <font color=black> Version 2 - Janvier 2024 </font>

##### <font color=grey> Version 1 - Prof. Alessandro L. Koerich.
##### Version 2 - Chargé de lab. Arthur Josi

Les laboratoires sont à faire par groupe de deux ou trois étudiants. Favorisez les groupes de trois.

| NOMS                  | CODE PERMANENT                                   |
|-----------------------|--------------------------------------------------|
| Étudiant1             | Code1                                            |
| Étudiant2             | Code2                                            |
| Étudiant3             | Code3                                            |

# Introduction

Ce premier laboratoire porte sur la préparation de données pour l'apprentissage machine. Le problème de classification qui vous est présenté est le problème [Facial Expression Recognition (FER)](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data), dont le but est de classer des visages dans sept catégories.

Veuillez noter que les images qui vous sont fournies ne sont pas nécessairement très faciles à travailler. Plusieurs images comportent du bruit, des artéfacts ou des éléments non pertinents. Le défi de ce laboratoire repose sur cette difficulté qui est chose courante dans les problèmes d’apprentissage machine moderne.

Voici, en exemple, des images de visages se retrouvant dans l’ensemble de données:

![Exemples de FER](https://miro.medium.com/max/2420/1*nXqJ4lMiBRp4Ilm3bpRxuA.png)

L’évaluation de ce laboratoire sera basée sur:
- la qualité des algorithmes proposés et utilisés;
- les réponses aux questions dans ce notebook;
- l'organisation de votre code source (SVP, n'oubliez pas de mettre des commentaires dans le code source!)

# Modules et bibliotèques python

### Import de bibliotèques

###  <font color=blue> À faire: </font>
N'oubliez pas d'ajouter une courte description aux bibliothèques que vous allez utiliser pour compléter ce notebook.

In [7]:
# !pip install imagehash facenet_pytorch

In [2]:
import numpy as np  # package for scientific computing with Python.
import matplotlib.pyplot as plt  # 2D plotting library
import cv2
from PIL import Image
import imagehash

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

# Partie 1 - Analyse exploratoire des données

On va commencer par regarder les données, c'est une pratique indispensable.

Pour ce lab, nous allons utiliser le dataset Facial Emotion Recognition (FER).

L'ensemble de données est disponible dans Moodle. Il contient presque 35,000 images de visages, avec une résolution de 48$\times$48 pixels en niveau de gris.

Les images se trouvent dans un fichier csv sous la forme d’un vecteur de 2,304 scalaires avec des valeurs entre 0 et 255.

Les partitions apprentissage, validation et test sont déjà préétablies.

Format du fichier: Emotion,Pixels,Usage avec:
- Emotion:  integer [0, 6]
- Pixels:   integer [0, 255]
- Usage:    string [Training, PublicTest, PrivateTest]

## Charger le fichier de données

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Load data
ferData = np.loadtxt('content/fer2013.csv', delimiter=',', dtype=str)

# Training partition
Xtrain = np.ones((28709, 2304), float)

for i in range(1, 28710):
    Xtrain[i - 1] = ferData[i, 1].split(" ")

ytrain = ferData[1:28710, 0].astype(int)

# Validation partition
Xval = np.ones((3589, 2304), float)

for i in range(28710, 32299):
    Xval[i - 28710] = ferData[i, 1].split(" ")

yval = ferData[28710:32299, 0].astype(int)

# Test partition
Xtest = np.ones((3589, 2304), float)

for i in range(32299, 35887):
    Xtest[i - 32299] = ferData[i, 1].split(" ")

ytest = ferData[32299:, 0].astype(int)

print(Xtrain.shape, Xval.shape, Xtest.shape)

In [ ]:
# Reshape les vecteurs de 2,304 dimensions vers matrices 48x48
# Afin d'avoir [samples][channels][width][height]

Xtrain = Xtrain.reshape(Xtrain.shape[0], 1, 48, 48).astype('uint8')
Xtest = Xtest.reshape(Xtest.shape[0], 1, 48, 48).astype('uint8')
Xval = Xval.reshape(Xval.shape[0], 1, 48, 48).astype('uint8')

print(Xtrain.shape, Xval.shape, Xtest.shape)

## <font color=black> 1a: Visualisation des images de visages </font>

###  <font color=blue> À faire: </font>

1. Créer une grille de dimension 7 lignes $\times$ $4$ colones avec des images de visage prises aleatoirement dans l'ensemble de apprentissage. Montrer une catégorie dans chaque ligne.

Vous pouvez visualiser les images en utilisant `plt.imshow`.


---
** Code ajouté **

In [ ]:
# Votre code ici
num_classes = 7
images_per_class = 4

emotion_labels = ["Colère", "Dégoût", "Peur", "Joie", "Tristesse", "Surprise",
                  "Neutre"]

fig, ax = plt.subplots(num_classes, images_per_class, figsize=(10, 15))

for class_id in range(num_classes):
    # Select only ids corresponding to the searched class
    idxs = np.where(ytrain == class_id)[0]
    selected = np.random.choice(idxs, images_per_class, replace=False)

    for j, img_idx in enumerate(selected):
        ax[class_id, j].imshow(Xtrain[img_idx, 0], cmap='gray')
        ax[class_id, j].axis('off')

        if j == 0:
            ax[class_id, j].set_ylabel(emotion_labels[class_id], fontsize=12,
                                       rotation=0, labelpad=50, va='center')

plt.suptitle("Images aléatoires par classe (1 ligne = 1 classe)", fontsize=16)
plt.subplots_adjust(left=0.2, hspace=0.4, top=0.93)
plt.show()

---

## 1b: Statistiques des données

Est-ce que vous avez un ensemble de données balancé? C.-à-d., une distribution égalitaire d’exemples par classe?

###  <font color=blue> À faire: </font>

1. Montrer les histogrammes de distribution des données pour les partitions de validation et test.

#### Code

In [ ]:
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']


def plot_class_histogram(y, title):
    fig, ax = plt.subplots()
    ax.hist(y, bins=np.arange(8) - 0.5,
            edgecolor='black')  # 7 bins: [-0.5, 0.5, ..., 6.5]

    ax.set_title(title)
    ax.set_xlabel("Class")
    ax.set_ylabel("Number of examples")

    # Met les noms de classe sous les barres
    ax.set_xticks(range(7))
    ax.set_xticklabels(labels, rotation=45, ha='right')

    plt.tight_layout()
    plt.show()


# Tracer les 3 histogrammes
plot_class_histogram(ytrain, "Distribution of classes - Training")
plot_class_histogram(yval, "Distribution of classes - Validation")
plot_class_histogram(ytest, "Distribution of classes - Test")


# Partie 2 - Préparation des données

## 2a: Nettoyage et normalisation des données

Avant de passer à d'autres étapes, vous devez vous assurer que il n'y a pas de:
- données abberantes;
- valeurs manquantes;
- valeurs inapplicables ou aberrantes;
- etc.   

###  <font color=blue> À faire: </font>

1. Concevoir un algorithme pour vérifier l'intégrité des données, faire des corrections si nécessaires, et finalement, normaliser les données dans la plage [0, 1].
2. Appliquer sur les ensembles d’apprentissage et validation. Attention! Étant donné que les données de l'ensemble de test sont considérées "inconnues" préalablement, il faut bien réfléchir quoi faire avec ces données.
3. Générer un fichier *fer2013-clean.csv* (même format) avec les données nettoyées et normalisées.

4. Décrire les étapes de votre algorithme/code.

#### Code

---
#### Intégrité des données

In [ ]:
# Check if there is NaN values in the data
assert not np.isnan(Xtrain).any(), "NaN détecté dans Xtrain"
assert not np.isnan(Xval).any(), "NaN détecté dans Xval"
assert not np.isnan(Xtest).any(), "NaN détecté dans Xtest"

In [ ]:
# Format checking
assert Xtrain.shape[1:] == (1, 48,
                            48), f"Xtrain shape incorrect: {Xtrain.shape}"
assert Xval.shape[1:] == (1, 48, 48), f"Xval shape incorrect: {Xval.shape}"
assert Xtest.shape[1:] == (1, 48, 48), f"Xtest shape incorrect: {Xtest.shape}"

assert Xtrain.dtype == np.uint8
assert Xtrain.min() >= 0 and Xtrain.max() <= 255

assert Xval.dtype == np.uint8
assert Xval.min() >= 0 and Xval.max() <= 255

assert Xtest.dtype == np.uint8
assert Xtest.min() >= 0 and Xtest.max() <= 255

---
#### Données abérrantes

**Affichage**

In [ ]:
def show_examples_grid(images, indices, title, max_samples=5):
    if len(indices) == 0:
        print(f"Aucune image détectée comme {title.lower()}.")
        return

    print(
        f"\n{len(indices)} images détectées comme {title.lower()}.\nAffichage des {min(len(indices), max_samples)} premières :")

    fig, axs = plt.subplots(1, min(len(indices), max_samples), figsize=(12, 2))
    fig.suptitle(title)
    for i, idx in enumerate(indices[:max_samples]):
        axs[i].imshow(images[idx], cmap='gray')
        axs[i].set_title(f"idx {idx}")
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()


def show_duplicates_grid(images, duplicates, max_samples=3):
    if len(duplicates) == 0:
        print("Aucun doublon détecté.")
        return

    print(
        f"\n{len(duplicates)} doublons détectés.\nAffichage des {min(len(duplicates), max_samples)} paires :")

    fig, axs = plt.subplots(len(duplicates[:max_samples]), 2,
                            figsize=(6, 2 * max_samples))
    for i, (idx1, idx2) in enumerate(duplicates[:max_samples]):
        axs[i, 0].imshow(images[idx1], cmap='gray')
        axs[i, 0].set_title(f"Image {idx1}")
        axs[i, 0].axis('off')

        axs[i, 1].imshow(images[idx2], cmap='gray')
        axs[i, 1].set_title(f"Doublon {idx2}")
        axs[i, 1].axis('off')
    plt.tight_layout()
    plt.show()

**Seuils**

In [ ]:
# determined by manually testing
THRESHOLD_DARK = 50
THRESHOLD_BRIGHT = 200
THRESHOLD_BLUR = 200.0

**Tests pour déterminer les seuils**

In [ ]:
variances = []
for img in Xtrain[:, 0]:  # shape (N, 48, 48)
    var = cv2.Laplacian(img, cv2.CV_64F).var()
    variances.append(var)

import matplotlib.pyplot as plt

plt.hist(variances, bins=50)
plt.title("Distribution de la netteté (Variance du Laplacien)")
plt.xlabel("Variance")
plt.ylabel("Nombre d'images")
plt.grid(True)
plt.show()

In [ ]:
# Calculate the variance for each image
laplacian_variances = []
indices_above_4000 = []

for idx, img in enumerate(Xtrain[:, 0]):  # images shape (N, 48, 48)
    var = cv2.Laplacian(img, cv2.CV_64F).var()
    laplacian_variances.append(var)
    if var < 200:
        indices_above_4000.append((idx, var))

# Show the detected images
max_samples = min(5, len(indices_above_4000))
if max_samples == 0:
    print("No image with sharpness greater than 4000")
else:
    print(f"{len(indices_above_4000)} images with sharpness> 4000")
    for i in range(max_samples):
        idx, var = indices_above_4000[i]
        plt.imshow(Xtrain[idx][0], cmap='gray')
        plt.title(f"idx={idx} — sharpness={var:.2f}")
        plt.axis('off')
        plt.show()


**Fonctions de vérification**

In [ ]:
def check_brightness(images):
    too_dark, too_bright = [], []
    for idx, img in enumerate(images):
        mean_val = img.mean()
        if mean_val < THRESHOLD_DARK:
            too_bright.append(idx)
        elif mean_val > THRESHOLD_BRIGHT:
            too_dark.append(idx)
    return too_dark, too_bright


def check_blurriness(images):
    blurry = []
    for idx, img in enumerate(images):
        laplacian_var = cv2.Laplacian(img, cv2.CV_64F).var()
        if laplacian_var < THRESHOLD_BLUR:
            blurry.append(idx)
    return blurry


def find_duplicates(images, hashfunc=imagehash.phash):
    seen = {}
    duplicates = []
    for idx, img in enumerate(images):
        pil_img = Image.fromarray(img)
        img_hash = hashfunc(pil_img)
        if img_hash in seen:
            duplicates.append((seen[img_hash], idx))
        else:
            seen[img_hash] = idx
    return duplicates

In [ ]:
# 1. Applati la dimension "channel"
Xtrain_images = Xtrain[:, 0]  # (n, 48, 48)

# 2. Exécute les détections
too_dark, too_bright = check_brightness(Xtrain_images)
blurry = check_blurriness(Xtrain_images)
dupes = find_duplicates(Xtrain_images)

# 3. Affiche les grilles organisées
show_examples_grid(Xtrain_images, too_dark, "Images trop sombres")
show_examples_grid(Xtrain_images, too_bright, "Images trop claires")
show_examples_grid(Xtrain_images, blurry, "Images floues")
show_duplicates_grid(Xtrain_images, dupes)



**Elimination des images qui ne sont pas des visages**

Tentative avec openCV

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
def preprocess_contrast(img):
    return cv2.equalizeHist(img)

def detect_faces(images, min_neighbors=3):
    detected_faces = []
    no_faces = []

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    for idx, img in enumerate(images):
        # S'assurer que l'image est en uint8 et 48x48
        img_uint8 = img.astype(np.uint8)
        img_eq = preprocess_contrast(img_uint8)
        img_resized = cv2.resize(img_eq, (96, 96), interpolation=cv2.INTER_LINEAR)
        faces = face_cascade.detectMultiScale(img_resized, scaleFactor=1.1, minNeighbors=3)


        if len(faces) > 0:
            detected_faces.append(idx)
        else:
            no_faces.append(idx)

    return detected_faces, no_faces


In [ ]:
Xtrain_images = Xtrain[:, 0]  # (n, 48, 48)
# has_face, no_face = detect_faces(Xtrain_images)

# print(f"{len(no_face)} images sans visage détecté")

# Affiche les images à supprimer (optionnel)
# show_examples_grid(Xtrain_images, no_face, "Images sans visage détecté", max_samples=5)

In [ ]:
def get_middle_line(img):
    h = img.shape[0]
    return img[h // 2, :]

In [ ]:
def plot_middle_line(img):
    line = get_middle_line(img)
    plt.figure(figsize=(6, 2))
    plt.plot(line)
    plt.title("Profil de la ligne médiane (niveaux de gris)")
    plt.xlabel("Colonne (x)")
    plt.ylabel("Intensité (0-255)")
    plt.grid(True)
    plt.show()

In [ ]:
def plot_middle_line_fft(img):
    line = get_middle_line(img)
    fft_vals = np.abs(np.fft.fft(line))
    freq = np.fft.fftfreq(len(line))

    plt.figure(figsize=(6, 2))
    plt.plot(freq[:len(freq)//2], fft_vals[:len(freq)//2])  # partie positive
    plt.title("FFT du profil de la ligne médiane")
    plt.xlabel("Fréquence")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

In [ ]:
img = Xtrain[42][0]  # par exemple

# Affiche l'image
plt.imshow(img, cmap='gray')
plt.title("Image choisie (idx=42)")
plt.axis('off')
plt.show()

# Affiche le profil + FFT
plot_middle_line(img)
plot_middle_line_fft(img)



In [ ]:
def analyze_fft_profile(profile):
    """Analyse la FFT du profil : retourne l'énergie, le ratio haute/basse fréquence et le spectre"""
    fft_vals = np.abs(np.fft.fft(profile))
    fft_vals = fft_vals[:len(fft_vals) // 2] # Keep only real part

    energy = np.sum(fft_vals ** 2)
    low_freq_energy = np.sum(fft_vals[:5] ** 2)
    high_freq_energy = np.sum(fft_vals[-10:] ** 2)
    ratio_high_low = high_freq_energy / (low_freq_energy + 1e-6)

    return energy, ratio_high_low, np.max(fft_vals)

In [ ]:
energy_list = []
ratio_list = []
peak_list = []

# Utilise un sous-échantillon pour accélérer (optionnel)
for img in Xtrain[:1000, 0]:  # Shape = (N, 48, 48)
    profile = get_middle_line(img)
    energy, ratio, peak = analyze_fft_profile(profile)
    energy_list.append(energy)
    ratio_list.append(ratio)
    peak_list.append(peak)

# --- Tracé 3D ---
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(energy_list, ratio_list, peak_list, c='blue', alpha=0.6)
ax.set_xlabel("Énergie totale du profil")
ax.set_ylabel("Ratio hautes/basses fréquences")
ax.set_zlabel("Pic FFT")
ax.set_title("Distribution des images selon leur profil spectral")
plt.tight_layout()
plt.show()

In [ ]:
def normalize_images_contrast(images):
    """
    Normalise le contraste de chaque image pour qu'elle occupe toute la plage [0, 255].
    images : tableau numpy (N, 48, 48) ou (N, 1, 48, 48)
    Retourne : tableau numpy (même shape), dtype uint8
    """
    # Gérer les images avec canal unique (N, 1, H, W)
    if images.ndim == 4 and images.shape[1] == 1:
        images = images[:, 0]  # (N, H, W)

    normed = np.empty_like(images, dtype=np.uint8)

    for i, img in enumerate(images):
        min_val = img.min()
        max_val = img.max()
        if max_val > min_val:
            scaled = (img - min_val) / (max_val - min_val) * 255
        else:
            scaled = np.zeros_like(img)
        normed[i] = scaled.astype(np.uint8)

    return normed


In [ ]:
Xtrain_norm = normalize_images_contrast(Xtrain)

energy_list = []
ratio_list = []
peak_list = []

# Utilise un sous-échantillon pour accélérer (optionnel)
for img in Xtrain[:1000, 0]:  # Shape = (N, 48, 48)
    profile = get_middle_line(img)
    energy, ratio, peak = analyze_fft_profile(profile)
    energy_list.append(energy)
    ratio_list.append(ratio)
    peak_list.append(peak)

# --- Tracé 3D ---
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(energy_list, ratio_list, peak_list, c='blue', alpha=0.6)
ax.set_xlabel("Énergie totale du profil")
ax.set_ylabel("Ratio hautes/basses fréquences")
ax.set_zlabel("Pic FFT")
ax.set_title("Distribution des images selon leur profil spectral")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib

matplotlib.pyplot.close()

#### Description

Votre description ici

## 2b: Prétraitement des images

**Point de départ:** *fer2013-clean.csv*

Il y a différents types de prétraitement que nous pouvons appliquer à des images dans les ensembles de données pour réduire la variabilité, réduire des bruits, etc.

En particulier, pour les images de visage, quelques prétraitements peuvent se montrer utiles, comme:
- Localisation/recadrage du visage.
- Localisation les yeux.
- Lissage du visage.
- Normalisation du contraste.
- Etc.

###  <font color=blue> À faire: </font>

1. Appliquer au moins un prétraitement sur les images de visages. Vous pouvez choisir différents algorithmes de prétraitement d’images dans [scikit-image](https://scikit-image.org/docs/stable/api/api.html). Vous pouvez aussi trouver d’autres types de prétraitement qui sont plus généraux dans [scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing).
2. Appliquer ce/ces algorithmes sur les ensembles d’apprentissage et validation. Attention! Étant donné que les données de l'ensemble de test sont considérées "inconnues" préalablement, il faut à nouveau bien réfléchir quoi faire avec ces données pour ne pas biaiser vos resultats, voir les rendre faux!
3. Générer un fichier *fer2013-clean-pre.csv* (même format) avec les données après le pretraitement.

4. Expliquer et justifier le prétraitement utilisé.


#### Code

In [ ]:
# Votre code ici

#### Description

Votre description ici

# Partie 3 - Classification

**Point de départ:** *fer2013-clean-pre.csv*

## 3a: Créer et évaluer une approche *template matching*

Un algorithme simple de classification consiste à calculer un modèle (gabarit/prototype/*template*) sur les données d'apprentissage, pour chaque classe (7 émotions = 7 modèles) et utiliser ces modèles (prototypes) pour faire des prédictions sur de nouvelles données.

Si les entrées ont toutes la même dimensionnalité (48x48), une façon très simple de calculer un modèle serait à partir du calcul des moyennes des valeurs de pixels) pour chaque classe.

Étant donné un prototype pour chaque classe (7 prototypes), nous pouvons classifier une nouvelle entrée (image de visage) en mesurant la distance (similarité ou dissimilarité) de telle image par rapport aux sept prototypes et en choisissant le prototype (la classe du prototype) le plus proche.

###  <font color=blue> À faire: </font>
1. Créer un *template*/gabarit/prototype pour chaque classe (émotion) depuis *fer2013-clean-pre.csv*
2. Afficher chaque prototype (visage moyen pour chaque classe)
3. Classifier par *template matching* (plus proche prototype), tous les exemples des ensembles d'apprentissage, validation et test) et rapporter les résultats suivants:<br>
3a. Rapport de classification produit avec *<font color=green>from sklearn.metrics import classification_report</font>*<br>
3b. taux de classification correct sur les trois (3) ensembles de données (sous la forme d'un tableau)<br>
3c. matrice de confusion produite avec *<font color=green> from sklearn.metrics import confusion_matrix</font>* pour les résultats sur l'ensemble de test (matrice 7 x 7 - étiquette x prédictions)
4. Faire une analyse des résultats et présenter vos conclusions sur l'approche *template matching* (Performance globale bonne/mauvaise, pourquoi? Performance par classe bonne/mauvaise pourquoi? Faiblesses et points fort, possibles améliorations, etc.)

#### Code

In [ ]:
# Votre code ici

#### Résultats et réponses

Vos réponses ici

Taux de classification (%) - Exemple:

| Ensemble | Modèle TM   |                   
|----------|-------------|
| Train      | 99,67       |                   
| Val      | 89,77       |                             
| Test     | 77,99       |        


# Partie 4 - Étude d'ablation

Point de départ: *fer2013-clean.csv*

Dans l'apprentissage machine, l'ablation est la suppression d'un composant d'un système d'apprentissage machine. Une étude d'ablation étudie les performances d'un système d'apprentissage machine en supprimant certains composants pour comprendre la contribution du composant au système global.

Alors, vous devez évaluer l'importance/contribution du prétraitement sur la performance.

###  <font color=blue> À faire: </font>
**Important: Aucun code requis pour la partie 4. Présentezr juste les résultats sous la forme d'un tableau comparatif.**
1. Refaire la Partie 3 avec *fer2013-clean.csv*
2. Rapportez les résultats suivants:<br>
2a. taux de classification correct sur les trois (3) ensembles de données (sous la forme d'un tableau)<br> 2b. matrice de confusion pour les résultats sur l'ensemble de test (matrice 7 x 7 - étiquettes x prédictions)
3. Faire une analyse des résultats et présenter vos conclusions sur l'importance du prétraitement.



#### Code
Aucun code requis

#### Résultats et résponses

Taux de classification correcte modèle TM (%)


| Ensemble | Avec prétraitement | Sans prétraitement |                                 
:-|:-:|-:
| App      |  99,67%       |   XX,XX%      |                   
| Val      |  89,77%       |   XX,XX%      |                             
| Test     |  77,99%       |   XX,XX%      |        


# Fin